In [2]:
# Generic
import typing
import os
from pathlib import Path
import typing
import html

# Plotting
import matplotlib.pyplot as plt

# Numerical Computing
import numpy as np
import torch

# Our Code
from maze_transformer.utils.notebook_utils import configure_notebook
from maze_transformer.dataset.maze_dataset import MazeDataset, MazeDatasetConfig
from maze_transformer.dataset.maze_dataset_configs import MAZE_DATASET_CONFIGS
from maze_transformer.training.config import ConfigHolder, ZanjHookedTransformer
from maze_transformer.evaluation.eval_model import evaluate_model
from maze_transformer.evaluation.baseline_models import RandomBaseline
from maze_transformer.evaluation.path_evals import PathEvals, PathEvalFunction

from maze_transformer.utils.utils import set_reproducibility, chunks, get_checkpoint_paths_for_run

In [ ]:
device = configure_notebook(seed=42, dark_mode=True)

In [ ]:
dataset: MazeDataset = MazeDataset.from_config(
    MAZE_DATASET_CONFIGS["test-g3-n5-a_dfs-h10466"], 
    local_base_path="../examples/maze_dataset/",
)